In [1]:
from sklearn.pipeline import Pipeline

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


import pickle
import yaml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import Dataset
from neuralnetworks import DynamicSeqRegNN, RegressionDataset

In [4]:
with open('./saved_models/Random Forest/rf_model.pkl', 'rb') as file:
        rf_model = pickle.load(file)

with open('./saved_models/XGBoost/xgb_model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)

In [5]:
from preprocess import Preprocessor

In [ ]:
# data operations Loading data
test = pd.read_table("../Data/test_FD001.txt", sep=' ', header=None)

p.rename_columns(train)
train = train.dropna(axis=1, how='all')
train = p.drop_no_info_cols(train)
train = p.create_rul_col(train)
sensor_cols, op_setting_cols, other_cols = p.update_col_groups(train)
train_norm = p.normalize(train, sensor_cols + op_setting_cols)
pca.fit(train_norm[predicting_cols])
train_test_split

In [ ]:
pipe_1 = Pipeline([
    
])